In [ ]:
# seam line group 이름 생성
stitch_dict.keys()
seam_line_groups = [f"SeamLinePairGroup_{idx}" for idx, _ in enumerate(stitch_dict.keys())]
print(seam_line_groups)

In [ ]:
# Seam Line Group 이름, pair 정보
SeamLinePairGroup = []
for idx, stitch in enumerate(stitch_dict.values()):
    SeamLinePairGroup.append(f"SeamLinePairGroup_{idx}")
print(SeamLinePairGroup)

['SeamLinePairGroup_0', 'SeamLinePairGroup_1', 'SeamLinePairGroup_2', 'SeamLinePairGroup_3', 'SeamLinePairGroup_4', 'SeamLinePairGroup_5', 'SeamLinePairGroup_6', 'SeamLinePairGroup_7', 'SeamLinePairGroup_8', 'SeamLinePairGroup_9', 'SeamLinePairGroup_10', 'SeamLinePairGroup_11', 'SeamLinePairGroup_12', 'SeamLinePairGroup_13', 'SeamLinePairGroup_14', 'SeamLinePairGroup_15', 'SeamLinePairGroup_16', 'SeamLinePairGroup_17', 'SeamLinePairGroup_18', 'SeamLinePairGroup_19', 'SeamLinePairGroup_20', 'SeamLinePairGroup_21', 'SeamLinePairGroup_22', 'SeamLinePairGroup_23']


In [ ]:
#TODO: 각 패널 엣지에 x, y 에 대한 좌표 정보를 얻기 위한 함수
import pandas as pd

def process_path_data(data):
    """
    주어진 Path 데이터를 DataFrame으로 처리.
    - 요소 타입 변환 (QuadraticBezier -> Bezier Curve, Line -> Straight)
    - start, control, control1, control2, end, radius의 실수부/허수부를 x, y로 분리
    - 복소수 열 제거
    
    Args:
        data (tuple): Path 객체와 속성 정보가 포함된 데이터
    
    Returns:
        pd.DataFrame: 처리된 데이터프레임
    """
    rows = []
    # Path 내부 요소 순회
    for element in data[0]:
        row = {
            "type": type(element).__name__,  # 요소 타입
            "start": getattr(element, "start", None),
            "controla": getattr(element, "control", None),
            "controlb": getattr(element, "control", None),
            "control1": getattr(element, "control1", None),
            "control2": getattr(element, "control2", None),
            "end": getattr(element, "end", None),
            "radius": getattr(element, "radius", None),
        }
        rows.append(row)
    
    # DataFrame 생성
    df = pd.DataFrame(rows)
    
    # type 열의 값을 변경하기 위한 매핑 사전
    type_mapping = {
        "QuadraticBezier": "Bezier Curve",
        "CubicBezier": "Bezier Curve",
        "Line": "Straight",
        "Arc": "Straight", #임의로 바꿈    
    }
    
    # type 열 값 변경
    df["type"] = df["type"].map(type_mapping).fillna(df["type"])
    
    # 대상 열 목록
    coordinate_columns = ["start",  "controla", "controlb","control1", "control2", "radius", "end"] 
    
    # 실수부와 허수부를 분리하여 새로운 열로 추가하는 함수
    def extract_coordinates(df, columns):
        for col in columns:
            df[f"{col}_x"] = df[col].apply(lambda z: z.real if isinstance(z, complex) else None)
            df[f"{col}_y"] = df[col].apply(lambda z: z.imag if isinstance(z, complex) else None)
        # TODO: y 좌표를 상하 반전
        max_y = df[['start_y', 'controla_y', 'controlb_y', 'control1_y', 'control2_y', 'end_y']].max().max()
        min_y = df[['start_y', 'controla_y', 'controlb_y', 'control1_y', 'control2_y', 'end_y']].min().min()
        stand_y = (max_y + min_y)
        for col in columns: 
            df[f"{col}_y"] = stand_y - df[f"{col}_y"]
        df['controlb_x'] = df['controla_x'].round(4) # TODO: ontrolb_x 에 round
        # TODO: 여기까지
        return df
    
    # 실수부와 허수부를 분리하여 데이터프레임에 추가
    df = extract_coordinates(df, coordinate_columns)
    
    # 기존 열 중 복소수 값을 제거
    df = df.drop(columns=coordinate_columns)
    
    return df

In [ ]:
# panel 이름 , edge 별 type, x, y 좌표, 
panel_names = list(pattern.panel_order())
for idx, panel_name in enumerate(panel_names):
    print(process_path_data(drawn_pattern_list[idx]))
    print(panel_name)

            type    start_x    start_y  control_x  control_y  control1_x  \
0   Bezier Curve  23.838050  37.467706  20.415804  37.467706         NaN   
1       Straight  17.003877  37.660943        NaN   0.000000         NaN   
2       Straight  15.351117  25.771222        NaN   0.000000         NaN   
3   Bezier Curve  15.228284  37.774638  12.956118  37.936948         NaN   
4       Straight  10.688537  38.185092        NaN   0.000000         NaN   
5       Straight   8.555268  27.594162        NaN   0.000000         NaN   
6   Bezier Curve   8.921332  38.391598   4.806185  38.960192         NaN   
7       Straight   0.711692  39.836598        NaN   0.000000         NaN   
8       Straight   0.000000  25.602731        NaN   0.000000         NaN   
9    CubicBezier   0.000000  20.989281        NaN   0.000000    7.776448   
10      Straight   6.609650   2.547801        NaN   0.000000         NaN   
11           Arc  13.019250   0.000000        NaN   0.000000         NaN   
12      Stra

In [ ]:
### 원래꺼 
# 이게 2024.01.14 12:58 + (15:55 기준 최신본.
# TODO: Pattern List + (InstanceDataList)에 대한 정보를 json으로 넣는 코드.
# ++ Bezier Curve, Straight가 아닌
# Arc , Qudratic Curve, Cubic Curve에 대해서는 어떻게 다룰지. 
# InstanceDataList 생성

# InstanceDataList 추가 루프

import pandas as pd
import json
import math

# 패널 데이터프레임 생성
panel_dataframes = [process_path_data(drawn_pattern) for drawn_pattern in drawn_pattern_list]
panel_names = list(pattern.panel_order())

# Initialize the top-level JSON structure
garment_data = {
    "FabricList": [],
    "GradingRuleTableList": [],
    "PatternList": [],
    "SymmetricDataList": [],
    "InstanceDataList": [],
    "SeamLinePairGroupList": [],
}

# Process each panel
for panel_name, df in zip(panel_names, panel_dataframes):
    # Initialize JSON structure for the panel
    panel_data = {
        "Name": panel_name,
        "fGrainlineAngle": 0.0,
        "ID": panel_name,
        "strSuperImposeSide": "None",
        "CurrentFabricUUID": "DWX6ey",
        "IsClosed": False,
        "InternalLineList": [],
        "ButtonHeadList": [],
        "ButtonHoleList": [],
        "AnnotationList": [],
        "NotchList": [],
        "IsHalfSymmetric": False,
        "ShapeInfo": {
            "IsSlashed": False,
            "LineList": [],
            "HalfSymmetryPointIDMap": {},
            "HalfSymmetryLineIDMap": {}
        },
        "ArrangementPointDataMap": {
                "PointName": "Body_Front_Center_3",
                "fOffSetX": 0.0,
                "fOffSetY": 0.05,
                "fAngle": 180.0
        }
    }
    # InstanceDataList에 json에 추가
    instance_data = {
        "OriginPatternID": panel_name,  # 패널 이름을 OriginPatternID로 사용
        "InstancePatternIDArray": []   # 빈 리스트로 추가
    }
    garment_data["InstanceDataList"].append(instance_data)
    # 패널별 좌표 ID 초기화
    coordinate_id_map = {}
    point_counter = 0  # 패널 내 좌표 카운터

    # Generate LineList and PointList for the panel
    for edge_idx, row in df.iterrows():
        line_id = f"{panel_name}_edge_{edge_idx}"
        point_list = []
        if row["type"] == "Bezier Curve":
            row_type = ["Straight", "Bezier Curve", "Bezier Curve", "Straight"]
            idx = 0
            for col_x, col_y in zip(row.index[1::2], row.index[2::2]):  # Skip 'type' and go by pairs
                x, y = row[col_x], row[col_y]
                if not (pd.isna(x) or pd.isna(y)):#TODO: or math.isclose(x, 0.0) and math.isclose(y, 0.0) ):
                    # 좌표에 대해 새로운 ID 생성 (패널 내에서만)
                    key = (x, y)
                    if key not in coordinate_id_map:
                        coordinate_id_map[key] = f"{panel_name}_point_{point_counter}"
                        point_counter += 1
                    point_id = coordinate_id_map[key]
                    
                    point_list.append({
                        "ID": point_id, 
                        "PointType": row_type[idx],
                        "Position": {"x": x, "y": y},
                        "GradingRuleID": -1
                    })
                    idx +=1        
        if row["type"] == "Straight":
            # Iterate through columns 2 by 2 for x and y
            for col_x, col_y in zip(row.index[1::2], row.index[2::2]):  # Skip 'type' and go by pairs
                x, y = row[col_x], row[col_y]
                if not (pd.isna(x) or pd.isna(y)):#TODO: or math.isclose(x, 0.0) and math.isclose(y, 0.0) ):
                    # 좌표에 대해 새로운 ID 생성 (패널 내에서만)
                    key = (x, y)
                    if key not in coordinate_id_map:
                        coordinate_id_map[key] = f"{panel_name}_point_{point_counter}"
                        point_counter += 1
                    point_id = coordinate_id_map[key]
                    
                    point_list.append({
                        "ID": point_id,
                        "PointType": row["type"],
                        "Position": {"x": x, "y": y},
                        "GradingRuleID": -1
                    })

        # Add Line to LineList
        line = {"ID": line_id, "PointList": point_list}
        panel_data["ShapeInfo"]["LineList"].append(line)

    # Add the panel data to the PatternList
    garment_data["PatternList"].append(panel_data)

# Convert to JSON string with indentation
json_output = json.dumps(garment_data, indent=4)

# Save to a file (optional)
output_file = "multiple_panels_data3.json"
with open(output_file, "w") as file:
    file.write(json_output)

# 결과 출력
print(json_output)

In [ ]:
# 각 panel에 대한 start, end 좌표를 저장해 놓는 함수
import math
from svgpathtools import Path, Line, QuadraticBezier, CubicBezier, Arc

def calculate_edge_length(edge):
    """
    각 엣지의 길이를 계산.
    """
    if isinstance(edge, Line):
        return abs(edge.end - edge.start)
    elif isinstance(edge, QuadraticBezier):
        return edge.length()
    elif isinstance(edge, CubicBezier):
        return edge.length()
    elif isinstance(edge, Arc):
        return edge.length()
    else:
        raise ValueError(f"Unsupported edge type: {type(edge)}")

def calculate_fstart_fend(path):
    """
    전체 엣지에 대해 fstart와 fend를 계산.
    """
    total_length = sum(calculate_edge_length(edge) for edge in path)
    fstart = 0.0
    results = []

    for idx, edge in enumerate(path):  # enumerate를 사용하여 번호 부여
        edge_length = calculate_edge_length(edge)
        fend = fstart + (edge_length / total_length)
        results.append({
            "edge_id": idx,  # 엣지 번호
            "edge": str(edge),  # Edge를 문자열로 변환하여 저장
            "fstart": round(fstart, 6),
            "fend": round(fend, 6)
        })
        fstart = fend

    return results

# 여러 패널의 정보를 저장할 딕셔너리
panel_fstart_fend_info = {}

# 패널 리스트 (예: pattern.panel_order())
panel_list = pattern.panel_order()

# 각 패널에 대해 fstart, fend 계산
for panel_name in panel_list:
    path = panel_svg_path_dict[panel_name][0]  # 패널에 해당하는 경로 정보 가져오기
    fstart_fend_results = calculate_fstart_fend(path)
    panel_fstart_fend_info[panel_name] = fstart_fend_results  # 결과를 딕셔너리에 저장

# panel_fstart_fend_info에 패널 정보, edge, start, end, edge_id 까지 저장 되어있다.
# 결과 출력 (딕셔너리 형태)
import pprint
pprint.pprint(panel_fstart_fend_info)

{'left_btorso': [{'edge': 'Line(start=37.46770632525742j, '
                          'end=10.034413661707j)',
                  'edge_id': 0,
                  'fend': 0.175079,
                  'fstart': 0.0},
                 {'edge': 'Arc(start=10.034413661707j, '
                          'radius=(10.849457592383715+10.849457592383715j), '
                          'rotation=0, large_arc=False, sweep=False, '
                          'end=(10.818800000000003+0j))',
                  'edge_id': 1,
                  'fend': 0.278636,
                  'fstart': 0.175079},
                 {'edge': 'Line(start=(10.818800000000003+0j), '
                          'end=(17.2284+2.5478006069423884j))',
                  'edge_id': 2,
                  'fend': 0.322656,
                  'fstart': 0.278636},
                 {'edge': 'CubicBezier(start=(17.2284+2.5478006069423884j), '
                          'control1=(16.844929+13.09047060694239j), '
                          'contr

In [ ]:
# 2024.01.14 14:33 기준 2:51 기준
# TODO: Seam Line Pair Group json으로 넣는 코드
# SeamLinePairGroupList 생성
garment_data["SeamLinePairGroupList"] = []
fabric_group = {
    "FabricName" : "Default Fabric",
    "FabricType" : "None",
    "FabricContent" : "None",
    "strBaseColorHexCode" : "#FFFFFF",
    "FabricUUID" : "DWX6ey"
    }
# SeamLinePairGroupList 생성 루프
for idx, value in enumerate(stitch_dict.values()):
    # "Name" 생성
    group_name = f"SeamLineGroup_{idx}"

    # value에서 첫 번째와 두 번째 ShapeID 정보 추출
    first_info = value[0]
    second_info = value[1]

    # 첫 번째 정보
    first_panel = first_info["panel"]
    first_edge = first_info["edge"]
    first_shape_id = f"{first_panel}_edge_{first_edge}"

    # 첫 번째 fStart와 fEnd 추출
    first_fstart_fend = next(
        (item for item in panel_fstart_fend_info[first_panel] if item["edge_id"] == first_edge),
        None
    )
    first_fstart = first_fstart_fend["fstart"]
    first_fend = first_fstart_fend["fend"]

    # 두 번째 정보
    second_panel = second_info["panel"]
    second_edge = second_info["edge"]
    second_shape_id = f"{second_panel}_edge_{second_edge}"

    # 두 번째 fStart와 fEnd 추출
    second_fstart_fend = next(
        (item for item in panel_fstart_fend_info[second_panel] if item["edge_id"] == second_edge),
        None
    )
    second_fstart = second_fstart_fend["fstart"]
    second_fend = second_fstart_fend["fend"]
    # Fabric default 구조 생성

    # JSON 구조 생성
    seam_line_pair_group = {
        "Name": group_name,
        "bIsTurned": False,
        "PairList": [
            {
                "First": {
                    "ShapeID": first_shape_id,
                    "LengthParam": {
                        "fStart": first_fstart,
                        "fEnd": first_fend
                    },
                    "Direction": True
                },
                "Second": {
                    "ShapeID": second_shape_id,
                    "LengthParam": {
                        "fStart": second_fstart,
                        "fEnd": second_fend
                    },
                    "Direction": True # TODO True로 일단 default
                }
            }
        ],
        "FoldData": {
            "iAngle": 180,
            "iStrength": 5
        }
    }

    # SeamLinePairGroupList에 추가
    garment_data["SeamLinePairGroupList"].append(seam_line_pair_group)

garment_data["FabricList"].append(fabric_group)    

# JSON 출력
json_output = json.dumps(garment_data, indent=4)

# Save to file (optional)
output_file = "garment_with_seam_lines.json"
with open(output_file, "w") as file:
    file.write(json_output)

# Print the result
print(json_output)

In [ ]:
# InstanceDataList json 넣는 코드

In [ ]:
# 젤 마지막에 진행 해야 할 부분.
# Initialize the top-level JSON structure
garment_data = {
    "FabricName": [],
    "GradingRuleTableList": [],
    "PatternList": [],
    "SymmetricDataList": [],
    "InstanceDataList": [],
    "SeamLinePairGroupList": [],
}

# 1. FabricName 정보 추가
garment_data["FabricName"] = ["DefaultFabric"]  # 여러 개의 패브릭 이름을 넣을 수 있습니다.

# 2. GradingRuleTableList 정보 추가
garment_data["GradingRuleTableList"] = [
    {
        "ID": "Rule1",
        "Description": "Default Grading Rule",
        "Details": []
    }
]

# 3. PatternList 정보 추가 (기존 코드 사용)
for panel_name, df in zip(panel_names, panel_dataframes):
    panel_data = {
        "Name": panel_name,
        "fGrainlineAngle": 0.0,
        "ID": panel_name,
        "strSuperImposeSide": "None",
        "CurrentFabricUUID": "",
        "IsClosed": False,
        "InternalLineList": [],
        "ButtonHeadList": [],
        "ButtonHoleList": [],
        "AnnotationList": [],
        "NotchList": [],
        "IsHalfSymmetric": False,
        "ShapeInfo": {
            "IsSlashed": False,
            "LineList": [],
            "HalfSymmetryPointIDMap": {},
            "HalfSymmetryLineIDMap": {}
        },
        "ArrangementPointDataMap": {
                "PointName": "Body_Front_Center_3",
                "fOffSetX": 0.0,
                "fOffSetY": 0.05,
                "fAngle": 0.0
        }
    }

    coordinate_id_map = {}
    point_counter = 0
    for edge_idx, row in df.iterrows():
        line_id = f"{panel_name}_edge_{edge_idx}"
        point_list = []
        for col_x, col_y in zip(row.index[1::2], row.index[2::2]):
            x, y = row[col_x], row[col_y]
            if not (pd.isna(x) or pd.isna(y) or math.isclose(x, 0.0) and math.isclose(y, 0.0)):
                key = (x, y)
                if key not in coordinate_id_map:
                    coordinate_id_map[key] = f"{panel_name}_point_{point_counter}"
                    point_counter += 1
                point_id = coordinate_id_map[key]
                point_list.append({
                    "ID": point_id,
                    "PointType": row["type"],
                    "Position": {"x": x, "y": y},
                    "GradingRuleID": -1
                })
        line = {"ID": line_id, "PointList": point_list}
        panel_data["ShapeInfo"]["LineList"].append(line)
    garment_data["PatternList"].append(panel_data)

# 4. SymmetricDataList 정보 추가
garment_data["SymmetricDataList"] = [
    {
        "ID": "Symmetry1",
        "Type": "Horizontal",
        "ReferencePanel": panel_names[0] if panel_names else None
    }
]

# 5. InstanceDataList 정보 추가
garment_data["InstanceDataList"] = [
    {
        "InstanceID": "Instance1",
        "PanelReference": panel_names[0] if panel_names else None,
        "Position": {"x": 0.0, "y": 0.0, "z": 0.0},
        "Rotation": {"x": 0.0, "y": 0.0, "z": 0.0}
    }
]

# 6. SeamLinePairGroupList 정보 추가
garment_data["SeamLinePairGroupList"] = [
    {
        "GroupID": "Seam1",
        "SeamLines": [
            {"StartPointID": "point_1", "EndPointID": "point_2"},
            {"StartPointID": "point_3", "EndPointID": "point_4"}
        ]
    }
]

# Convert to JSON string with indentation
json_output = json.dumps(garment_data, indent=4)

# Save to a file (optional)
output_file = "complete_garment_data.json"
with open(output_file, "w") as file:
    file.write(json_output)

# 결과 출력
print(json_output)

In [ ]:
#TODO: 진짜로 다 삭제 됐을때 답없을때 백업해놓으려고 복사한 부분 2024.01.20 16:53 작성

In [ ]:
# TODO:아래에 해당하는 arc_to_bezier는 개구려서 새로 만든거임
import numpy as np
import re

def arc_to_single_control_bezier(arc):
    """
    Arc 데이터를 단일 Control Point를 가진 Bezier Curve로 변환합니다.
    
    Args:
        arc (dict): Arc 데이터 (start_x, start_y, end_x, end_y, radius, large_arc, sweep 포함).
    
    Returns:
        dict: Bezier Curve의 시작점, 끝점, 제어점 1개.
    """
    
    # 시작점과 끝점 정의
    start_x, start_y = arc["start_x"], arc["start_y"]
    end_x, end_y = arc["end_x"], arc["end_y"]
    radius = arc["radius"]  # 반지름
    large_arc = arc["large_arc"]
    sweep = arc["sweep"]

    # 시작점과 끝점을 복소수로 변환
    start = np.array([start_x, start_y])
    end = np.array([end_x, end_y])

    # 중심 계산
    center = (start + end) / 2
    cx, cy = center[0], center[1]

    # 각도 계산
    theta_start = np.arctan2(start_y - cy, start_x - cx)
    theta_end = np.arctan2(end_y - cy, end_x - cx)

    # sweep 및 large_arc 처리
    if not sweep:
        theta_start, theta_end = theta_end, theta_start
    if large_arc and abs(theta_end - theta_start) < np.pi:
        theta_end += 2 * np.pi

    # 중간 각도 계산
    theta_m = (theta_start + theta_end) / 2  # 중간 각도

    # Control Point 계산 (중간 각도 기준)
    control_x = cx + radius * np.cos(theta_m)
    control_y = cy + radius * np.sin(theta_m)
    control_point = np.array([control_x, control_y])

    # Control Point를 직선(start-end) 기준으로 대칭 변환
    def reflect_point_over_line(p, p0, p1):
        # 직선 벡터
        line_vec = p1 - p0
        line_vec /= np.linalg.norm(line_vec)  # 정규화
        # 점과 직선의 수직 대칭 계산
        proj = np.dot(p - p0, line_vec) * line_vec
        reflection = 2 * proj - (p - p0)
        return p0 + reflection

    # Control Point 대칭
    control_reflected = reflect_point_over_line(control_point, start, end)

    # 반환 형식
    return {
        "start_x": start_x, "start_y": start_y,
        "control1_x": control_reflected[0], "control1_y": control_reflected[1],
        "control2_x": control_reflected[0].round(4), "control2_y": control_reflected[1],
        "end_x": end_x, "end_y": end_y,
    }

def transform_3d_point(fstart_fend):
    start_3d_point = None  # 기본값 설정
    end_3d_point = None
    panel_name = fstart_fend["panel"]
    edge_str= fstart_fend["edge"]
    pattern = r'start=([^\s,]+).*end=([^\s,]+)'
    match = re.search(pattern, edge_str)
    print("match", match)
    if match:
        start_str = match.group(1)  # "8.92133247524254+33.21647810564835j"
        end_str   = match.group(2)  # "0.7116916666666668+34.66147782685373j"
        print(f"{start_str}, {end_str}, start_str, end_str")
        
        
        # 불필요한 문자 제거
        start_str = start_str.strip("(")
        end_str = end_str.strip("(")
        start_str = start_str.strip(")")
        end_str = end_str.strip(")")
        print("start_str:", start_str, "end_str:", end_str)

        # 문자열 -> 복소수 변환
        start_complex = complex(start_str)  # (8.92133247524254+33.21647810564835j)
        end_complex   = complex(end_str)    # (0.7116916666666668+34.66147782685373j)

        # 실수부, 허수부 분리
        start_point = (start_complex.real, start_complex.imag) # start_y
        end_point = (end_complex.real, end_complex.imag) # end_y
        print(start_point, end_point)
        print("start_point type", type(start_point))
        #TODO: 2024.01.20
        translation = translations_and_rotations[panel_name]['translation']
        rotation = translations_and_rotations[panel_name]['rotation']
        print("translation", translation)
        print("rotation", rotation)
        
        
        
        start_3d_point = transform_2d_points_to_3d(start_point, translation, rotation)
        end_3d_point = transform_2d_points_to_3d(end_point, translation, rotation)
        
        return start_3d_point, end_3d_point
 
 
# make panel direction 그냥 둠 2024.01.20
# def select_direction_back(fstart_fend):

#     panel_name = fstart_fend["panel"]
#     edge_str= fstart_fend["edge"]
#     pattern = r'start=\(([^)]+)\).*end=\(([^)]+)\)'
#     match = re.search(pattern, edge_str)
#     if match:
#         start_str = match.group(1)  # "8.92133247524254+33.21647810564835j"
#         end_str   = match.group(2)  # "0.7116916666666668+34.66147782685373j"
        
#         # 문자열 -> 복소수 변환
#         start_complex = complex(start_str)  # (8.92133247524254+33.21647810564835j)
#         end_complex   = complex(end_str)    # (0.7116916666666668+34.66147782685373j)

#         # 실수부, 허수부 분리
#         start_point = (start_complex.real, start_complex.imag) # start_y
#         end_point = (end_complex.real, end_complex.imag) # end_y
        
#         #TODO: 2024.01.20
#         translation = translations_and_rotations[panel_name]['translation']
#         rotation = translations_and_rotations[panel_name]['rotation']

#         start_3d_point = transform_2d_points_to_3d(start_point, translation, rotation)
#         end_3d_point = transform_2d_points_to_3d(end_point, translation, rotation)
        
#     return start_3d_point, end_3d_point            


# 3차원으로 변환
import numpy as np

def euler_to_rotation_matrix(rx_deg, ry_deg, rz_deg):
    """
    Rx, Ry, Rz (degree) 오일러 각을 받아서
    X축 -> Y축 -> Z축 순서로 회전하는 3x3 회전 행렬을 반환.
    """
    # 각도를 radian으로 변환
    rx = np.deg2rad(rx_deg)
    ry = np.deg2rad(ry_deg)
    rz = np.deg2rad(rz_deg)

    # 회전 행렬 구성

    # 1) X축 회전 행렬
    Rx = np.array([
        [1,          0,           0],
        [0,  np.cos(rx), -np.sin(rx)],
        [0,  np.sin(rx),  np.cos(rx)]
    ])

    # 2) Y축 회전 행렬
    Ry = np.array([
        [ np.cos(ry), 0, np.sin(ry)],
        [          0, 1,          0],
        [-np.sin(ry), 0, np.cos(ry)]
    ])

    # 3) Z축 회전 행렬
    Rz = np.array([
        [np.cos(rz), -np.sin(rz), 0],
        [np.sin(rz),  np.cos(rz), 0],
        [         0,           0, 1]
    ])

    # 순서: X축 회전 -> Y축 회전 -> Z축 회전 (Intrinsic)
    # 최종 회전 행렬 = Rz * Ry * Rx (행렬 곱 순서 유의)
    R = Rz @ Ry @ Rx

    return R

def transform_2d_points_to_3d(points_2d, translation, rotation):
    """
    2D 좌표 리스트(points_2d)를 rotation, translation 정보를 사용해
    3D 좌표로 변환.
    
    Parameters
    ----------
    points_2d : list of (x, y)
        변환 대상 2D 좌표들
    translation : list or tuple
        [Tx, Ty, Tz] 형태
    rotation : list or tuple
        [Rx_deg, Ry_deg, Rz_deg] 형태 (도 단위)
    
    Returns
    -------
    transformed_points_3d : np.ndarray
        변환된 3D 좌표 (N x 3)
    """
    if isinstance(points_2d, tuple) and len(points_2d) == 2:
        x, y = points_2d
    else:
        raise ValueError("Expected a single 2D coordinate as a tuple (x, y).")

    # 2D -> 3D 확장 (z=0)
    points_3d = np.array([x, y, 0.0])    
    print("points_3d", points_3d)
    # 2) 오일러 각 -> 회전 행렬
    Rx_deg, Ry_deg, Rz_deg = rotation
    R = euler_to_rotation_matrix(Rx_deg, Ry_deg, Rz_deg)

    # 3) 회전 적용
    # 행렬 곱: (N x 3) · (3 x 3) = (N x 3)
    points_rotated = points_3d @ R.T

    # 4) 평행이동 적용
    Tx, Ty, Tz = translation
    points_translated = points_rotated + np.array([Tx, Ty, Tz])

    return points_translated

In [ ]:
# 1번
#TODO: 각 패널 엣지에 x, y 에 대한 좌표 정보를 얻기 위한 함수
import pandas as pd

def process_path_data(data):
    """
    주어진 Path 데이터를 DataFrame으로 처리.
    - 요소 타입 변환 (QuadraticBezier -> Bezier Curve, Line -> Straight)
    - start, control, control1, control2, end, radius의 실수부/허수부를 x, y로 분리
    - 복소수 열 제거
    
    Args
        data (tuple): Path 객체와 속성 정보가 포함된 데이터
    
    Returns:
        pd.DataFrame: 처리된 데이터프레임
    """
    rows = []
    # Path 내부 요소 순회
    for element in data[0]:
        row = {
            "type": type(element).__name__,  # 요소 타입
            "start": getattr(element, "start", None),
            "controla": getattr(element, "control", None),
            "controlb": getattr(element, "control", None),
            "control1": getattr(element, "control1", None),
            "control2": getattr(element, "control2", None),
            "end": getattr(element, "end", None),
            "radius": getattr(element, "radius", None),
            "rotation": getattr(element, "rotation", None),
            "large_arc": getattr(element, "large_arc", None),
            "sweep": getattr(element, "sweep", None),
        }
        rows.append(row)
    
    # DataFrame 생성
    df = pd.DataFrame(rows)
    
    # type 열의 값을 변경하기 위한 매핑 사전
    type_mapping = {
        "QuadraticBezier": "Bezier Curve",
        "CubicBezier": "Bezier Curve",
        "Line": "Straight",
        "Arc": "Arc", #임의로 바꿈    
    }
    
    # type 열 값 변경
    df["type"] = df["type"].map(type_mapping).fillna(df["type"])
    
    # TODO: 2024.01.16 이후 Arc에 대한 정보에 대해 Bezier Curve로 변환예정.
    # -> 일단 되게 끔 하는게 목표.
    # 대상 열 목록
    coordinate_columns = ["start",  "controla", "controlb", "control1", "control2",  "end"] 
    
    # 실수부와 허수부를 분리하여 새로운 열로 추가하는 함수
    def extract_coordinates(df, columns):
        for col in columns:
            df[f"{col}_x"] = df[col].apply(lambda z: z.real if isinstance(z, complex) else None)
            df[f"{col}_y"] = df[col].apply(lambda z: z.imag if isinstance(z, complex) else None)
        # TODO: x, y 좌표를 상하 반전
        # minimize
        max_y = df[['start_y', 'controla_y', 'controlb_y', 'control1_y', 'control2_y', 'end_y']].max().max()
        min_y = df[['start_y', 'controla_y', 'controlb_y', 'control1_y', 'control2_y', 'end_y']].min().min()
        min_x = df[['start_x', 'controla_x', 'controlb_x', 'control1_x', 'control2_x', 'end_x']].min().min()
        max_x = df[['start_x', 'controla_x', 'controlb_x', 'control1_x', 'control2_x', 'end_x']].max().max()
        
        stand_y = (max_y + min_y)
        stand_x = (max_x + min_x)
        for col in columns: 
            df[f"{col}_y"] = (stand_y - df[f"{col}_y"]) * 10 #TODO: 01.17 scaling까지 
            df[f"{col}_x"] = df[f"{col}_x"]* 10 #TODO: scaling까지 
        df['controlb_x'] = df['controla_x'].round(4) # TODO: ontrolb_x 에 round
        # TODO: 여기까지
        return df
    
    # 실수부와 허수부를 분리하여 데이터프레임에 추가
    df = extract_coordinates(df, coordinate_columns)
    
    # 기존 열 중 복소수 값을 제거
    df = df.drop(columns=coordinate_columns)
    
    return df

In [ ]:
# 2번
# 각 panel에 대한 start, end 좌표를 저장해 놓는 함수
import math
from svgpathtools import Path, Line, QuadraticBezier, CubicBezier, Arc
# TODO: 2024.01.19 15:29 name 추가함.
def calculate_edge_length(edge):
    """
    각 엣지의 길이를 계산.
    """
    if isinstance(edge, Line):
        return abs(edge.end - edge.start)
    elif isinstance(edge, QuadraticBezier):
        return edge.length()
    elif isinstance(edge, CubicBezier):
        return edge.length()
    elif isinstance(edge, Arc):
        return edge.length()
    else:
        raise ValueError(f"Unsupported edge type: {type(edge)}")

def calculate_fstart_fend(path, panel_name):
    """
    전체 엣지에 대해 fstart와 fend를 계산.
    """
    total_length = sum(calculate_edge_length(edge) for edge in path)
    fstart = 0.0
    results = []

    for idx, edge in enumerate(path):  # enumerate를 사용하여 번호 부여
        edge_length = calculate_edge_length(edge) # TODO: name 넣어봄 01.19.17:00
        fend = fstart + (edge_length / total_length)
        results.append({
            "edge_id": idx,  # 엣지 번호
            "edge": str(edge),  # Edge를 문자열로 변환하여 저장
            "fstart": round(fstart, 6),
            "fend": round(fend, 6)
        })
        fstart = fend

    return results

# 여러 패널의 정보를 저장할 딕셔너리
panel_fstart_fend_info = {}

# 패널 리스트 (예: pattern.panel_order())
panel_list = pattern.panel_order()

# 각 패널에 대해 fstart, fend 계산
for panel_name in panel_list:
    path = panel_svg_path_dict[panel_name][0]  # 패널에 해당하는 경로 정보 가져오기
    fstart_fend_results = calculate_fstart_fend(path, panel_name) # TODO: 2024.01.19 14:33 panel_name 넣기
    panel_fstart_fend_info[panel_name] = fstart_fend_results  # 결과를 딕셔너리에 저장

# panel_fstart_fend_info에 패널 정보, edge, start, end, edge_id 까지 저장 되어있다.
# 결과 출력 (딕셔너리 형태)
import pprint
pprint.pprint(panel_fstart_fend_info)

{'skirt_back': [{'edge': 'QuadraticBezier(start=(7.175201096088408+41.976578529428835j), '
                         'control=(5.3194177995763425e-06+29.6022786833952j), '
                         'end=24.393471210275766j)',
                 'edge_id': 0,
                 'fend': np.float64(0.055537),
                 'fstart': 0.0},
                {'edge': 'QuadraticBezier(start=24.393471210275766j, '
                         'control=(1.939242881832115e-07+12.153931313590752j), '
                         'end=(2.105173726856094+0j))',
                 'edge_id': 1,
                 'fend': np.float64(0.126637),
                 'fstart': np.float64(0.055537)},
                {'edge': 'Line(start=(2.105173726856094+0j), '
                         'end=(10.360811974887312+0j))',
                 'edge_id': 2,
                 'fend': np.float64(0.150581),
                 'fstart': np.float64(0.126637)},
                {'edge': 'Line(start=(10.360811974887312+0j), '
                 

In [ ]:
panel_svg_path_dict

{'skirt_back': (Path(QuadraticBezier(start=(7.175201096088408+41.976578529428835j), control=(5.3194177995763425e-06+29.6022786833952j), end=24.393471210275766j),
       QuadraticBezier(start=24.393471210275766j, control=(1.939242881832115e-07+12.153931313590752j), end=(2.105173726856094+0j)),
       Line(start=(2.105173726856094+0j), end=(10.360811974887312+0j)),
       Line(start=(10.360811974887312+0j), end=(11.774477794543008+18.764645557214386j)),
       Line(start=(11.774477794543008+18.764645557214386j), end=(13.188143614198705+0j)),
       Line(start=(13.188143614198705+0j), end=(17.74669356111238+0j)),
       Line(start=(17.74669356111238+0j), end=(19.160359380768078+20.860844903977508j)),
       Line(start=(19.160359380768078+20.860844903977508j), end=(20.574025200423772+0j)),
       Line(start=(20.574025200423772+0j), end=(34.24967516043286+0j)),
       Line(start=(34.24967516043286+0j), end=(35.66334098008855+20.860844903977508j)),
       Line(start=(35.66334098008855+20.860

In [ ]:
# 3번
### 원래꺼 
# 이게 2024.01.14 12:58 + (15:55 기준 최신본.
# TODO: Pattern List + (InstanceDataList)에 대한 정보를 json으로 넣는 코드.
# ++ Bezier Curve, Straight가 아닌
# Arc , Qudratic Curve, Cubic Curve에 대해서는 어떻게 다룰지. 
# InstanceDataList 생성

# InstanceDataList 추가 루프

import pandas as pd
import json
import math

# 패널 데이터프레임 생성
panel_dataframes = [process_path_data(drawn_pattern) for drawn_pattern in drawn_pattern_list]
panel_names = list(pattern.panel_order())

# Initialize the top-level JSON structure
garment_data = {
    "FabricList": [],
    "GradingRuleTableList": [],
    "PatternList": [],
    "SymmetricDataList": [],
    "InstanceDataList": [],
    "SeamLinePairGroupList": [],
}

# Process each panel
for panel_name, df in zip(panel_names, panel_dataframes):
    # Initialize JSON structure for the panel
    panel_data = {
        "Name": panel_name,
        "fGrainlineAngle": 0.0,
        "ID": panel_name,
        "strSuperImposeSide": "None",
        "CurrentFabricUUID": "DWX6ey",
        "IsClosed": False,
        "InternalLineList": [],
        "ButtonHeadList": [],
        "ButtonHoleList": [],
        "AnnotationList": [],
        "NotchList": [],
        "IsHalfSymmetric": False,
        "ShapeInfo": {
            "IsSlashed": False,
            "LineList": [],
            "HalfSymmetryPointIDMap": {},
            "HalfSymmetryLineIDMap": {}
        },
        "ArrangementPointDataMap": {
                "PointName": "Body_Front_Center_3",
                "fOffSetX": 0.0,
                "fOffSetY": 0.05,
                "fAngle": 180.0
        }
    }
    # InstanceDataList에 json에 추가
    instance_data = {
        "OriginPatternID": panel_name,  # 패널 이름을 OriginPatternID로 사용
        "InstancePatternIDArray": []   # 빈 리스트로 추가
    }
    garment_data["InstanceDataList"].append(instance_data)
    # 패널별 좌표 ID 초기화
    coordinate_id_map = {}
    point_counter = 0  # 패널 내 좌표 카운터

    # Generate LineList and PointList for the panel
    for edge_idx, row in df.iterrows():
        line_id = f"{panel_name}_edge_{edge_idx}"
        point_list = []
        columns_x = [col for col in row.index if col.endswith('_x')]
        columns_y = [col for col in row.index if col.endswith('_y')]
        
        if row["type"] == "Bezier Curve":
            row_type = ["Straight", "Bezier Curve", "Bezier Curve", "Straight"]
            idx = 0
            for col_x, col_y in zip(columns_x, columns_y):  # Skip 'type' and go by pairs
                x, y = row[col_x], row[col_y]
                if not (pd.isna(x) or pd.isna(y)):#TODO: or math.isclose(x, 0.0) and math.isclose(y, 0.0) ):
                    # 좌표에 대해 새로운 ID 생성 (패널 내에서만)
                    key = (x, y)
                    if key not in coordinate_id_map:
                        coordinate_id_map[key] = f"{panel_name}_point_{point_counter}"
                        point_counter += 1
                    point_id = coordinate_id_map[key]
                    
                    point_list.append({
                        "ID": point_id, 
                        "PointType": row_type[idx],
                        "Position": {"x": x, "y": y},
                        "GradingRuleID": -1
                    })
                    idx +=1 
        # TODO: Arc 타입에 대해 베지어로 변환하여 Json에 넣는 작업.
        elif row["type"] == "Arc":
            arc_bezier = arc_to_bezier(row)
            columns_x = [key for key in row.keys() if key.endswith('_x')]
            columns_y = [key for key in row.keys() if key.endswith('_y')]
            # 사용하지 않을 키 필터링
            excluded_keys = {"controla_x", "controlb_x", "controla_y", "controlb_y"}
            columns_x = [col for col in columns_x if col not in excluded_keys]
            columns_y = [col for col in columns_y if col not in excluded_keys]
            print(columns_x, columns_y)
            
            row_type = ["Straight", "Bezier Curve", "Bezier Curve", "Straight"]
            idx = 0
            for col_x, col_y in zip(columns_x, columns_y):  # Skip 'type' and go by pairs
                print(arc_bezier[col_x], arc_bezier[col_y])
                x, y = arc_bezier[col_x], arc_bezier[col_y]
                if not (pd.isna(x) or pd.isna(y)):#TODO: or math.isclose(x, 0.0) and math.isclose(y, 0.0) ):
                    # 좌표에 대해 새로운 ID 생성 (패널 내에서만)
                    key = (x, y)
                    if key not in coordinate_id_map:
                        coordinate_id_map[key] = f"{panel_name}_point_{point_counter}"
                        point_counter += 1
                    point_id = coordinate_id_map[key]
                    
                    point_list.append({
                        "ID": point_id, 
                        "PointType": row_type[idx],
                        "Position": {"x": x, "y": y},
                        "GradingRuleID": -1
                    })
                    idx +=1          
        elif row["type"] == "Straight":
            # Iterate through columns 2 by 2 for x and y
            for col_x, col_y in zip(columns_x, columns_y):  # Skip 'type' and go by pairs
                x, y = row[col_x], row[col_y]
                if not (pd.isna(x) or pd.isna(y)):#TODO: or math.isclose(x, 0.0) and math.isclose(y, 0.0) ):
                    # 좌표에 대해 새로운 ID 생성 (패널 내에서만)
                    key = (x, y)
                    if key not in coordinate_id_map:
                        coordinate_id_map[key] = f"{panel_name}_point_{point_counter}"
                        point_counter += 1
                    point_id = coordinate_id_map[key]
                    
                    point_list.append({
                        "ID": point_id,
                        "PointType": row["type"],
                        "Position": {"x": x, "y": y},
                        "GradingRuleID": -1
                    })
    

        # Add Line to LineList
        line = {"ID": line_id, "PointList": point_list}
        panel_data["ShapeInfo"]["LineList"].append(line)

    # Add the panel data to the PatternList
    garment_data["PatternList"].append(panel_data)

# # Convert to JSON string with indentation
# json_output = json.dumps(garment_data, indent=4)

# # Save to a file (optional)
# output_file = "multiple_panels_data3.json"
# with open(output_file, "w") as file:
#     file.write(json_output)

# # 결과 출력
# print(json_output)

In [ ]:
# 4번
# 2024.01.14 14:33 기준 2:51 기준
# 2024.01.14 TODO: Seam Line Pair Group json으로 넣는 코드
# 2024.01.16 TODO: front , back 에 따라 Pair List의 Direction 결정 넣기
# SeamLinePairGroupList 생성
####TODO: 여기부터 내가 임의로 테스트 해봄 2024.01.19 16:16
file_path = '/media/hjp/db6095ca-a560-4c3a-90ad-b667ec189671/REFERENCES/3D_VTO/GarmentCode/GarmentCode/ANALYSIS_josef/nique_panel_names.csv'
data = pd.read_csv(file_path)

# 열 이름 설정
data.columns = ['name']

# 분류 로직 추가
data['classification'] = data['name'].apply(
    lambda x: 'back' if '_back' in x or '_b' in x or '_btorso' in x else
            'front' if '_front' in x or '_f' in x or '_ftorso' in x else
            'other'
)

# 각 분류에 따른 리스트 생성
front_list = data[data['classification'] == 'front']['name'].tolist()
back_list = data[data['classification'] == 'back']['name'].tolist()
other_list = data[data['classification'] == 'other']['name'].tolist()    
#### TODO: 여기까지    

garment_data["SeamLinePairGroupList"] = []
fabric_group = {
    "FabricName" : "Default Fabric",
    "FabricType" : "None",
    "FabricContent" : "None",
    "strBaseColorHexCode" : "#FFFFFF",
    "FabricUUID" : "DWX6ey"
    }
# SeamLinePairGroupList 생성 루프
for idx, value in enumerate(stitch_dict.values()):
    

    
    # "Name" 생성
    group_name = f"SeamLineGroup_{idx}"

    # value에서 첫 번째와 두 번째 ShapeID 정보 추출
    first_info = value[0]
    second_info = value[1]
        
    # 정보 저장
    first_panel = first_info["panel"] # 패널 정보
    first_edge = first_info["edge"]
    first_shape_id = f"{first_panel}"
    
    second_panel = second_info["panel"]
    second_edge = second_info["edge"]
    second_shape_id = f"{second_panel}" #TODO: 이부분 제외 '_edge_{second_edge}"'
    
    # 첫 번째 fStart와 fEnd 추출
    first_fstart_fend = next(
        (item for item in panel_fstart_fend_info[first_panel] if item["edge_id"] == first_edge),
        None
    )
    # 두 번째 fStart와 fEnd 추출
    second_fstart_fend = next(
        (item for item in panel_fstart_fend_info[second_panel] if item["edge_id"] == second_edge),
        None
    )
    
    print(first_fstart_fend)
    print(second_fstart_fend)
    # first, second panel 정보 넣음
    if first_fstart_fend is not None:
        first_fstart_fend["panel"] = first_panel
    first_point = first_fstart_fend["edge"]
    
    if second_fstart_fend is not None:
        second_fstart_fend["panel"] = second_panel
    second_point = second_fstart_fend["edge"]
    # TODO 2024.01.19 14:30 direeciton 넣기 

    first_start_point, first_end_point = transform_3d_point(first_fstart_fend)
    second_start_point, second_end_point = transform_3d_point(second_fstart_fend)
    
    first_vec = first_end_point - first_start_point
    second_vec = second_end_point - second_start_point
    
    dot_product = np.dot(first_vec, second_vec)
    
    

    if (first_panel or second_panel) in front_list:
        first_direction = True
    
    # if second_panel in _list:
    #     second_direction = True
        

    
    
    # TODO: 기존에 해놓은 정보 그대로 둬봄 2024.01.20.13:52
    # if first_panel in front_list:
    #     first_direction = transform_3d_point(first_fstart_fend)
    # elif first_panel in back_list:
    #     first_direction = transform_3d_point(first_fstart_fend)
    # else:
    #     first_direction = False
    
    # if second_panel in front_list:
    #     second_direction = transform_3d_point(second_fstart_fend)
    # elif second_panel in back_list:
    #     second_direction = transform_3d_point(second_fstart_fend)
    # else:
    #     second_direction = False

    # 패널 내부에서 붙는 경우의 조건식 
    if (first_panel == second_panel) or ():
        first_direction = True
        second_direction = False
    else:
        pass
    
    
    first_direction = True
    second_direction = False
    
    # direction 결정 뒤에 정해 놓는 fstart, fend
    if first_direction == False:
        first_fstart, first_fend = first_fstart_fend["fend"], first_fstart_fend["fstart"]
    else:
        first_fstart, first_fend = first_fstart_fend["fstart"], first_fstart_fend["fend"]
    
    # direction 결정 뒤에 정해 놓는 fstart, fend        
    if second_direction == False:
        second_fstart, second_fend = second_fstart_fend["fend"], second_fstart_fend["fstart"]
    else:
        second_fstart, second_fend = second_fstart_fend["fstart"], second_fstart_fend["fend"]
    

    # JSON 구조 생성
    seam_line_pair_group = {
        "Name": group_name,
        "bIsTurned": False,
        "PairList": [
            {
                "First": {
                    "ShapeID": first_shape_id,
                    "LengthParam": {
                        "fStart": first_fstart,
                        "fEnd": first_fend
                    },
                    "Direction": first_direction
                },
                "Second": {
                    "ShapeID": second_shape_id,
                    "LengthParam": {
                        "fStart": second_fstart,
                        "fEnd": second_fend
                    },
                    "Direction": second_direction 
                }
            }
        ],
        "FoldData": {
            "iAngle": 180,
            "iStrength": 5
        }
    }

    # SeamLinePairGroupList에 추가
    garment_data["SeamLinePairGroupList"].append(seam_line_pair_group)

garment_data["FabricList"].append(fabric_group)    

# JSON 출력
json_output = json.dumps(garment_data, indent=4)

# Save to file (optional)
output_file = "no_arc_data_0120_1438_v3.json"
with open(output_file, "w") as file:
    file.write(json_output)

# Print the result
print(json_output)

{'edge_id': 3, 'edge': 'Line(start=(10.360811974887312+0j), end=(11.774477794543008+18.764645557214386j))', 'fstart': np.float64(0.150581), 'fend': np.float64(0.20516)}
{'edge_id': 4, 'edge': 'Line(start=(11.774477794543008+18.764645557214386j), end=(13.188143614198705+0j))', 'fstart': np.float64(0.20516), 'fend': np.float64(0.259738)}
match <re.Match object; span=(5, 81), match='start=(10.360811974887312+0j), end=(11.7744777945>
(10.360811974887312+0j), (11.774477794543008+18.764645557214386j)), start_str, end_str
start_str: 10.360811974887312+0j end_str: 11.774477794543008+18.764645557214386j
(10.360811974887312, 0.0) (11.774477794543008, 18.764645557214386)
start_point type <class 'tuple'>
translation [-26.36872837467025, 29.170279842593317, -20.0]
rotation [0.0, 0.0, 0.0]
points_3d [10.36081197  0.          0.        ]
points_3d [11.77447779 18.76464556  0.        ]
match <re.Match object; span=(5, 81), match='start=(11.774477794543008+18.764645557214386j), e>
(11.774477794543008+1